In [1]:
from bertopic import BERTopic
import pandas as pd
import numpy as np
import os

In [2]:
# import data
# data_df = pd.read_csv('C:\\Users\\gcmar\\Desktop\\GIT_REPOS\\LAB\\Literature_summary\\Papers\\scopus_bark_ambrosia_beetles.csv')
# data_df['text'] = data_df['Title'].apply(str) + data_df['Author Keywords'].apply(str) + data_df['Index Keywords'].apply(str) + data_df['Abstract'].apply(str) 
# data_df['topic_text'] = data_df['Title'].apply(str) + data_df['Author Keywords'].apply(str) + data_df['Index Keywords'].apply(str)
# docs = data_df['text'].dropna().tolist()
# topic_docs = data_df['topic_text'].dropna().tolist()

# # file_lst = data_df['file'].unique()
# for j in file_lst:
#     try:
#         print('_______________________________\n\n', j)
#         df = data_df[data_df['file']==j]
#         for i in range(start_point, len(df), win_size):
#             coref_series = coref_res(text_series=df["text"].iloc[i:i+win_size])
#             print('coref done', i, '-', i+win_size)
#             link_entities_series = link_entities(text_series=coref_series)
#             print('entity linking done', i, '-', i+win_size)
#             entities_df = pd.concat(link_entities_series.tolist())
#             print('df create done', i, '-', i+win_size)
#             entities_df_lst.append(entities_df)
#             print('df to list done', i, '-', i+win_size, '\n')
#         all_entities_df = pd.concat(entities_df_lst)
#         all_entities_df.reset_index(drop=True, inplace=True)
#         edge_lst_df = all_entities_df.value_counts().reset_index().rename(columns={0: "count"})
#         edge_lst_df.to_csv('entity_weighted_edgelist_ALL_'+j[:-4]+'.csv')
#         edge_lst_df.to_feather('entity_weighted_edgelist_ALL_'+j[:-4]+'.feather')
#     except:
#         print("DID NOT WORK WITH - ", j)

pwd = os.getcwd()
data_path = os.path.dirname(pwd) + "\\Data\\NDPs\\"
data_df = pd.read_feather(data_path+"Africa_docs.feather")
data_df = data_df.drop_duplicates().reset_index(drop=True)
data_df["text"] = data_df["text"].astype(str)
data_df['text'] = data_df['text'].str.strip()
data_df['text'] = data_df['text'].str.split('. \n') # OR '. \n'
data_df = data_df.explode('text').reset_index(drop=True)
# docs = data_df['text'].dropna().tolist()

In [3]:
emb_model = "multi-qa-mpnet-base-dot-v1" # "all-mpnet-base-v2" OR multi-qa-mpnet-base-dot-v1  OR "xlm-r-bert-base-nli-stsb-mean-tokens"
model = BERTopic(embedding_model=emb_model, nr_topics="auto", top_n_words=15, min_topic_size=3) 

#### Topics per doc

In [4]:
finished_lst = []
# for file in os.listdir(os.getcwd()):
#     filename = os.fsdecode(file)
#     if filename.endswith(".html"):
#         finished_lst.append(filename)
        
file_lst = data_df['file'].unique()       
for i in file_lst:
    if i[:-4]+'_'+emb_model+'_topics.html' in finished_lst:
        print(i, "FOUND", "\n_________________\n")
    else:
        print(i, "STARTED")
        
        # select only text from one pdf
        docs = data_df[data_df['file']==i]['text'].str.strip().dropna().tolist()
        # decrease the max number of words in items in docs
        print("fixing doc lengths")
        # new_docs = []
        # for str_val in docs:
        #     if len(str_val.split(" ")) > 512:
        #         new_docs.append(str_val.split("\n"))
        #     else:
        #         new_docs.append(str_val)
        # new_docs_flat = [item for sublist in new_docs for item in sublist]
        # docs = new_docs_flat
        
        # increase number of items in docs if number is too small
        if len(docs)<20:
            docs_new = []
            for str_val in docs:
                docs_new += str_val.split('\n')
                docs = docs_new
        # estiamte topics and probabilities
        print("applying model")
        try:
            model = BERTopic(embedding_model=emb_model, nr_topics="auto", top_n_words=15, min_topic_size=3)
            topics, probabilities = model.fit_transform(docs)
        except: 
            model = BERTopic(embedding_model=emb_model, nr_topics=10, top_n_words=15, min_topic_size=3)
            topics, probabilities = model.fit_transform(docs)
            print("reduced topics as text was too small")
        # get topic counts and probabilities in one dataframe
        topic_prob_df = pd.DataFrame([topics, probabilities]).T
        topic_prob_mean_df = topic_prob_df.groupby([0]).mean(1).reset_index().sort_values(0)
        topic_count_s = topic_prob_df[0].value_counts().sort_index()
        topic_prob_mean_df['count'] = topic_count_s.to_list()
        topic_prob_mean_df.columns = ['topic', 'probability', 'count']
        # add topic words column
        topic_df = pd.DataFrame(np.array(list(model.topic_representations_.values()))[:,:,0])
        topic_df['topic_words'] = topic_df[topic_df.columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
        topic_df['topic'] = list(model.topic_representations_)
        topic_df['topic_words'][0] = None
        topic_df = topic_df[['topic', 'topic_words']]
        topic_prob_mean_df = topic_prob_mean_df.merge(topic_df,how='left', on='topic')
        # create visualization
        try:
            fig = model.visualize_topics()
            fig.write_html(i[:-4]+'_'+emb_model+'_topics.html')
            topic_prob_mean_df.to_csv(i[:-4]+'_'+emb_model+'_topics.csv', index=False)
        except:
            print("data size too small to run properly")
            topic_prob_mean_df.to_csv(i[:-4]+'_'+emb_model+'_topics_TooShort.csv', index=False)
        print(i, "DONE", "\n_________________\n")

Botswana NDP.pdf STARTED
fixing doc lengths
applying model
Botswana NDP.pdf DONE 
_________________

Botswana VNR.pdf STARTED
fixing doc lengths
applying model
Botswana VNR.pdf DONE 
_________________

Cameroon MDP.pdf STARTED
fixing doc lengths
applying model
Cameroon MDP.pdf DONE 
_________________

Cameroon NDP.pdf STARTED
fixing doc lengths
applying model
Cameroon NDP.pdf DONE 
_________________

Eswatini NDP.pdf STARTED
fixing doc lengths
applying model
Eswatini NDP.pdf DONE 
_________________

Eswatini VNR.pdf STARTED
fixing doc lengths
applying model
data size too small to run properly
Eswatini VNR.pdf DONE 
_________________

Gambia NDP.pdf STARTED
fixing doc lengths
applying model
data size too small to run properly
Gambia NDP.pdf DONE 
_________________

Gambia VNR.pdf STARTED
fixing doc lengths
applying model
Gambia VNR.pdf DONE 
_________________

Ghana MDP.pdf STARTED
fixing doc lengths
applying model
Ghana MDP.pdf DONE 
_________________

Ghana NDP.pdf STARTED
fixing doc 

In [5]:
#######################################################################################

In [6]:
# data_df['topic'] = topics
# data_df['topic_prob'] = probabilities.tolist()

# # add topic words column
# topic_df = pd.DataFrame(np.array(list(model.topic_representations_.values()))[:,:,0])
# topic_df['topic_words'] = topic_df[topic_df.columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
# topic_df['topic'] = list(model.topic_representations_)
# topic_df['topic_words'][0] = None
# topic_df = topic_df[['topic', 'topic_words']]
# data_df.merge(topic_df,how='left', on='topic')

# data_df.to_csv('papers_bertopic.csv', index=False)
# data_df.to_feather('papers_bertopic.feather', index=False)

# model.get_topic_info()

# fig = model.visualize_topics()
# fig.write_html("topics_viz.html")

#### Topics over time

In [7]:
# topics_over_time = model.topics_over_time(docs, timestamps=data_df['Year'].tolist())

In [8]:
# fig = model.visualize_topics_over_time(topics_over_time)
# fig.write_html("topics_over_time.html")

#### Multiple topics

In [9]:
# model = BERTopic(embedding_model="all-mpnet-base-v2", nr_topics="auto", top_n_words=5) #"xlm-r-bert-base-nli-stsb-mean-tokens"

In [10]:
# num_repeats = 5
# for i in range(num_repeats):
#     topics, probabilities = model.fit_transform(docs)
#     data_df['topic_'+str(i)] = topics
#     data_df['topic_prob_'+str(i)] = probabilities.tolist()

#     # add topic words column
#     topic_df = pd.DataFrame(np.array(list(model.topic_representations_.values()))[:,:,0])
#     topic_df['topic_words_'+str(i)] = topic_df[topic_df.columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
#     topic_df['topic'] = list(model.topic_representations_)
#     topic_df['topic_words_'+str(i)][0] = None
#     topic_df = topic_df[['topic', 'topic_words_'+str(i)]]
#     data_df = data_df.merge(topic_df,how='left', left_on='topic_'+str(i), right_on='topic')

In [11]:
# data_df.to_csv('papers_bertopics_multi.csv', index=False)